In [83]:
import pandas as pd

In [84]:
import numpy as np

In [85]:
crop_df = pd.read_csv(r'D:\ECO221_Group-25\data\ECO221_Project_2025_Final.csv')

In [86]:
rainfall_df = pd.read_csv(r'D:\ECO221_Group-25\data\RF_DistrictWise_ECO221_2025.csv')

In [87]:
salinity_df = pd.read_csv(r'D:\ECO221_Group-25\data\Salinity_Alkalinity_ECO221_2025.csv')

In [88]:
sugarcane_df = crop_df[crop_df['crop'].str.lower() == 'sugarcane'].copy()

In [89]:
#Capping the irrigated Area
sugarcane_df['irrigatedarea_capped'] = sugarcane_df[["irrigatedarea1000hectares", "area1000hectares"]].min(axis=1)

In [90]:
sugarcane_df['irrigatedarea_new'] = sugarcane_df['irrigatedarea_capped'] + 1

In [91]:
sugarcane_df['unirrigated_area'] = sugarcane_df['area1000hectares'] - sugarcane_df['irrigatedarea_capped']


In [114]:
initial_districts = crop_df[crop_df["crop"].str.lower() == "sugarcane"]["districtname"].str.lower().str.strip().nunique()
print("Initial sugarcane districts:", initial_districts)

Initial sugarcane districts: 266


In [92]:
rainfall_2017 = rainfall_df[rainfall_df['yearcode'] == 2017].copy()
rainfall_2017['month'] = rainfall_2017['month'].str.lower().str.strip()
rainfall_2017['district'] = rainfall_2017['district'].str.lower().str.strip()

In [93]:
monsoon_months = ['june, 2017', 'july, 2017', 'august, 2017', 'september, 2017']


In [94]:
# === Aggregate annual rainfall ===
annual_rainfall = rainfall_2017.groupby('district')['rf'].sum().reset_index()
annual_rainfall.columns = ['districtname', 'annual_rainfall_mm']

In [95]:
# === Aggregate monsoon rainfall ===
monsoon_rainfall = rainfall_2017[rainfall_2017['month'].isin(monsoon_months)]
monsoon_summary = monsoon_rainfall.groupby('district')['rf'].sum().reset_index()
monsoon_summary.columns = ['districtname', 'monsoon_rainfall_mm']


In [96]:
sugarcane_df['districtname'] = sugarcane_df['districtname'].str.lower().str.strip()
annual_rainfall['districtname'] = annual_rainfall['districtname'].str.lower().str.strip()
monsoon_summary['districtname'] = monsoon_summary['districtname'].str.lower().str.strip()

In [97]:
# === Merge rainfall summaries ===
sugarcane_df = pd.merge(sugarcane_df, annual_rainfall, on='districtname', how='left')
sugarcane_df = pd.merge(sugarcane_df, monsoon_summary, on='districtname', how='left')

In [100]:
print("Merged sugarcane data with rainfall:")
print(sugarcane_df[['districtname', 'annual_rainfall_mm', 'monsoon_rainfall_mm']].head(10))

Merged sugarcane data with rainfall:
  districtname  annual_rainfall_mm  monsoon_rainfall_mm
0         durg          861.629998           805.980000
1       bastar          902.189990           781.249990
2       raipur          683.989988           621.329990
3     bilaspur         1857.730027          1539.510022
4      raigarh          624.520000           563.340000
5      surguja          580.889988           516.769989
6     jabalpur          826.810001           775.400000
7     balaghat          962.159988           899.959990
8   chhindwara          953.030709           882.499790
9  narsinghpur          817.670000           771.730000


In [115]:
merged_districts_task1 = sugarcane_df['districtname'].nunique()
missing_rain = sugarcane_df[sugarcane_df['annual_rainfall_mm'].isna() | sugarcane_df['monsoon_rainfall_mm'].isna()]['districtname'].unique()
print("Merged districts (Task 1):", merged_districts_task1)
print("Districts missing rainfall data:", len(missing_rain))

Merged districts (Task 1): 266
Districts missing rainfall data: 48


In [113]:
sugarcane_df.to_csv('data/task1_sugarcane_with_rainfall.csv', index=False)

In [102]:
salinity_df.head()

,District Name,Average Salinity/Alkalinity (Percentage)
0,South Andamans,0.000000
1,Nicobars,0.000000
2,North And Middle Andaman,0.000000
3,Anantapur,12.341882
4,Chittoor,2.652205


In [105]:
salinity_df['districtname']= salinity_df['District Name'].str.lower().str.strip()

In [106]:
sugarcane_df['districtname'] = sugarcane_df['districtname'].str.lower().str.strip()

In [108]:
sugarcane_df = pd.merge(
    sugarcane_df,
    salinity_df[['districtname', 'Average Salinity/Alkalinity (Percentage)']],  # adjust if column name is different
    on='districtname',
    how='left'
)

In [109]:
print(sugarcane_df[['districtname', 'Average Salinity/Alkalinity (Percentage)']].dropna().head())


  districtname  Average Salinity/Alkalinity (Percentage)
0         durg                                  0.000000
1       bastar                                  0.000000
2       raipur                                  0.000000
3     bilaspur                                  0.390813
4      raigarh                                  0.000000


In [111]:
print(sugarcane_df['Average Salinity/Alkalinity (Percentage)'].notnull().sum(), "districts with salinity data")


217 districts with salinity data


In [112]:
# Save the DataFrame after merging salinity data
sugarcane_df.to_csv('data/task2_sugarcane_salinity_merged.csv', index=False)